<a href="https://colab.research.google.com/github/Smladenovska/vnp/blob/master/196065_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 23.9MB/s]


In [18]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/content/ElectricCarData.csv')

In [5]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    object 
 1   Model            103 non-null    object 
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    object 
 7   RapidCharge      103 non-null    object 
 8   PowerTrain       103 non-null    object 
 9   PlugType         103 non-null    object 
 10  BodyStyle        103 non-null    object 
 11  Segment          103 non-null    object 
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 11.4+ KB


In [7]:
data.isnull().sum()

,0
Brand,0
Model,0
AccelSec,0
TopSpeed_KmH,0
Range_Km,0
Efficiency_WhKm,0
FastCharge_KmH,0
RapidCharge,0
PowerTrain,0
PlugType,0


In [8]:
data.replace('-', pd.NA, inplace=True)

In [11]:
num_col = [col for col in data.columns if data[col].dtype == 'int64' or data[col].dtype  == 'float64']
if 'FastCharge_KmH' not in num_col:
    num_col.append('FastCharge_KmH')
num_col

['AccelSec',
 'TopSpeed_KmH',
 'Range_Km',
 'Efficiency_WhKm',
 'Seats',
 'PriceEuro',
 'FastCharge_KmH']

In [13]:

for col in num_col:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [14]:
data.fillna(data.mean(numeric_only=True), inplace=True)

In [15]:
print(data.dtypes)

Brand               object
Model               object
AccelSec           float64
TopSpeed_KmH         int64
Range_Km             int64
Efficiency_WhKm      int64
FastCharge_KmH     float64
RapidCharge         object
PowerTrain          object
PlugType            object
BodyStyle           object
Segment             object
Seats                int64
PriceEuro            int64
dtype: object


In [16]:
cat_cols = [col for col in data.columns if data[col].dtype == 'object']
cat_cols = [col for col in cat_cols if col != 'FastCharge_KmH']
cat_cols

['Brand',
 'Model',
 'RapidCharge',
 'PowerTrain',
 'PlugType',
 'BodyStyle',
 'Segment']

In [19]:
label_encoders = {}
for col in cat_cols:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])


Split the dataset for training and testing in ratio 80:20

In [20]:
X = data.drop('PriceEuro',axis=1)
y = data['PriceEuro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Initialize the DecisionTreeRegressor model, and use the fit function for training the model.**

In [21]:
dtr_model = DecisionTreeRegressor(max_depth=10, min_samples_split=5, max_features='sqrt')
dtr_model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10, max_features='sqrt', min_samples_split=5)

**Predict the outcomes for X test**

In [23]:
dtr_pred= dtr_model.predict(X_test)

## **Assess the model performance, by using sklearn metrics for regression**

In [25]:
dt_mae = mean_absolute_error(y_test, dtr_pred)
dt_mse = mean_squared_error(y_test, dtr_pred)
dt_r2 = r2_score(y_test, dtr_pred)

print("DecisionTreeRegressor Performance:")
print(f"Mean Absolute Error: {dt_mae}")
print(f"Mean Squared Error: {dt_mse}")
print(f"R-squared: {dt_r2}")

DecisionTreeRegressor Performance:
Mean Absolute Error: 7640.099206349206
Mean Squared Error: 104300827.27546294
R-squared: 0.8684793493754546



# **Initialize the XGBRegressor model, and use the fit function**

In [27]:
xgb_model = XGBRegressor(n_estimators = 100, max_depth=6, learning_rate=0.1, objective="reg:squarederror")
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)


# Predict the outcomes for X test

In [28]:
xgb_pred = xgb_model.predict(X_test)

# **Assess the model performance, by using sklearn metrics for regression**

In [29]:
xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)

print("\nXGBRegressor Performance:")
print(f"Mean Absolute Error: {xgb_mae}")
print(f"Mean Squared Error: {xgb_mse}")
print(f"R-squared: {xgb_r2}")


XGBRegressor Performance:
Mean Absolute Error: 7674.605840773809
Mean Squared Error: 229197751.91820925
R-squared: 0.7109875679016113


# **Compare the performances of both model for at least three regression metircs**

In [30]:
print("\nComparison of Model Performances:")
print(f"DecisionTreeRegressor - MAE: {dt_mae}, MSE: {dt_mse}, R2: {dt_r2}")
print(f"XGBRegressor - MAE: {xgb_mae}, MSE: {xgb_mse}, R2: {xgb_r2}")


Comparison of Model Performances:
DecisionTreeRegressor - MAE: 7640.099206349206, MSE: 104300827.27546294, R2: 0.8684793493754546
XGBRegressor - MAE: 7674.605840773809, MSE: 229197751.91820925, R2: 0.7109875679016113



MAE = XGBRegressor lower better
MSE = XGBRegressor lower better
R2 = XGBRegressor higher better
XGBRegressor is better model.